**Professor:** Enrique Garcia Ceja
**email:** enrique.gc@tec.mx

# Exercise: Activity recognition with a CNN and transfer learning.

In this exercise you will train a CNN to classify human acitivities from accelerometer sensors collected with a smartphone. You will be using the human activity recognition WISDM dataset (http://www.cis.fordham.edu/wisdm/dataset.php) which contains 6 different activities: walking, jogging, walking upstairs, walking downstairs, sitting and standing. The data was collected with an Android phone located in the participants' pants pocket. You will also use transfer learning to fine tune your model and adapt it to a specific user.

The data was collected with a sampling rate of 20Hz (1 sample every 50ms). The accelerometer sensor captures values from three orientations (x,y and z). The picture below shows an example of acceleration values over time while a participant was walking.

<table><tr><td><img src="https://github.com/enriquegit/ap-img/blob/main/img/cel.png?raw=true" width="200"></td><td><img src="https://github.com/enriquegit/ap-img/blob/main/img/walking.png?raw=true" width="400"></td></tr></table>

From the picture above, it can be seen that we are dealing with time series data. But how can we train a CNN with this type of data? You can transform your data into an image-like representation! The timeseries data can be shaped into a matrix-like structure. See image below. In the example, the timeseires have a length of 9 and they are reshaped into 3x3 matrices. This can be seen as a RGB image (red, green, blue) but in this case it is a XYZ image! The time series values will be treated as if they were image pixels.

<img src="https://github.com/enriquegit/ap-img/blob/main/img/representation.png?raw=true" width="750">

The dataset has already been converted into image-like representations. The timeseries were split into five seconds segments (non-overlapping). Since the sampling rate was 20Hz, we will have arrays of length 100 for each x, y and z. The arrays will be reshaped into 10x10 images. Since we have x, y and z, each instance will be an image with dimension 10x10x3. Each row in the file images.txt contains the five second segments for all x, y and z values flattened. That is, the first 100 values correspond to x, the next 100 values to y and the last 100 values to z. The last two columns are the userid and label, respcectively.

**In this exercise you will:**

* Select a target user and build a model for her/him.
* Train a general (user independent) CNN model.
* Use 50% of the target user data as test set. Use the remaining 50% to build an adaptive model by fine tuning the general model using transfer learning.


## Part 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import Model

from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Path to the dataset.
filepath = "datasets/images.txt"

# We will save the trained CNN model into a file. Specify the path where the model will be saved.
savemodelpath = "datasets/" # Make sure the directory already exists in your local computer.

# Define the size of the images.
img_width = 10
img_height = 10
img_depth = 3

# Read the dataset.
dataset = pd.read_csv(filepath)

In [ ]:
# Display the first 5 rows of the dataset.
dataset.head()

In [ ]:
# Select features 'pixels' (acceleration values), userids,and class.
pixels = dataset.drop(['userid','label'], axis=1)
userids = dataset[['userid']]
labels = dataset[['label']]

# Convert to numpy.
pixels = pixels.values
userids = userids.values
labels = labels.values

# Encode the labels from strings to integers.
le = LabelEncoder()

labels = le.fit_transform(labels.ravel())

# Reshape pixels into images of 10x10x3.
images = np.reshape(pixels, (pixels.shape[0],img_width,img_height,img_depth), order='F')

## Part 2

Now the `images` variable contains all our 10x10x3 images. The `userids` and `labels` variables contain the user ids and activity labels, respectively.

In [ ]:
# Print the shapes of our data.
print(userids.shape)
print(labels.shape)
print(images.shape)

In [ ]:
# Shuffle the data.

def shuffle_instances(features, userids, labels):
    n = features.shape[0]
    np.random.seed(1234)
    idxs = np.random.choice(n, size=n, replace=False)
    features = features[idxs,:,:,:]
    userids = userids[idxs]
    labels = labels[idxs]

    return(features, userids, labels)

images, userids, labels = shuffle_instances(images, userids, labels)

Let's plot some of the images! The following code will print some warnings since the images are not exactly between 0-255 because strictly speaking, they are not images but the visualizations will still be useful. We can see that for low energy activities such as sitting or standing, the colors look very solid. On the other hand, for the jogging activity, the images look more fragmented.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
    plt.xlabel(le.inverse_transform(labels)[i])
plt.show()

In [ ]:
# Print the available user ids.
with np.printoptions(threshold=np.inf):
    print(np.unique(userids))

In [ ]:
# Select a target user from the above ids. This is the user we will use to test the general model.
# We will also adapt the general model to this target user.

target_user =  ### COMPLETE THE CODE ###

In [ ]:
# Split into training test sets.
# The train set will be the data from all users except target_user.
# The train set will be stored in images_all and labels_all.
# The test set will be the data that belongs to target_user and is stored in images_target and labels_target.

# Get indices
idxs_target = np.where(userids == target_user)[0]
idxs_all = np.where(userids != target_user)[0]

images_all = images[idxs_all]
labels_all = labels[idxs_all]

images_target = images[idxs_target]
labels_target = labels[idxs_target]

In [ ]:
# Normalize images using the maximum value from the training data (images_all).

# First, get the maximum value from images_all.
# The max() function from numpy returns the maximum value of a numpy array.
maxval =  #### COMPLETE THE CODE ####

# Normalize images_all using maxval.
images_all =  #### COMPLETE THE CODE ####

# Normalize images_target using maxval.
images_target =  #### COMPLETE THE CODE ####

## Part 3

Now we need a train and test set for the target user. The training set is used to fine tune the general model. We will use 50% of the target user's data to adapt the general model and the remaining to test the performance.

Randomly split the target_user data (`images_target` `labels_target`) and assign 50% of the images and labels into variables defined as `images_target_train` and `labels_target_train` respectively. Store the remaining 50% of the images and labels in `images_target_test` and `labels_target_test`. Tip: You can use the `train_test_split()` function from scikit learn.

In [ ]:
#### YOUR CODE HERE ####


In [ ]:
# Print the dimensions of our datasets.
print(images_target_train.shape)
print(labels_target_train.shape)

print(images_target_test.shape)
print(labels_target_test.shape)

## Part 4

Now build a CNN model. Since the images are small, you are advised to use a small kernel size, e.g., (2,2). Remember that you can use dropout to reduce overfitting.

In [ ]:
# Define your CNN general model here using keras functional API and name it 'model'.
# The model takes as input images and predicts the activity type.
# You should assign a unique name to each layer with the 'name' attribute.
# This will later allow you to select specific layers by name.

#### YOUR CODE HERE ####





In [ ]:
# Print a summary of your model.
model.summary()

In [ ]:
# Define an optimizer and its learning rate.
optimizer =  #### COMPLETE THE CODE ####

# Set the batch size.
batch_size =   #### COMPLETE THE CODE ####

# Set number of epochs.
num_epochs =   #### COMPLETE THE CODE ####

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# The general model is fitted with data from all users except the target user (images_all, labels_all).

#### COMPLETE THE CODE ####
history = model.fit(, ,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    validation_split=0.15,
                    verbose=1)

# Save model after it has been fitted.
model.save(savemodelpath + 'my_model.keras')

In [ ]:
# plot history

import matplotlib.pyplot as plt
%matplotlib inline
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Evaluate the model on the target user's test data and print accuracy.
# You can use the model.evaluate() function.

#### YOUR CODE HERE ####



## Part 5

### Apply transfer learning

Now, let's load the general model and adapt it to the target user.

In [ ]:
# Load the general model previously trained.
model = keras.models.load_model(savemodelpath + 'my_model.keras')
model.summary()

In [ ]:
# Freeze some of the layers of the genral model to apply transfer learning.
# Each layer has a 'trainable' attribute that you can set to False.
# When you re-fit a model, layers whose trainable attribute is False will be untouched.
# You can retrieve a layer with model.get_layer("layerName")

#### YOUR CODE HERE ####



# Define an optimizer and its learning rate.
optimizer2 = keras.optimizers.SGD(learning_rate=0.01)

# Compile the model again to apply changes.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer2,
              metrics=['accuracy'])

In [ ]:
# Print summary.
model.summary()

## Part 6

After freezing some layers you can note that the number of trainable parameters was reduced. Now let's fine tune the model with the target user's train set: `images_target_train` `labels_target_train`.

In [ ]:
#### COMPLETE THE CODE ####
# Since the target train set is very small, you can set validation_split = 0.0

history = model.fit(, ,
                    batch_size = batch_size,
                    epochs = 15, # You can change this as well.
                    validation_split = ,
                    verbose = 1)

In [ ]:
# Evaluate the adapted model on the test set (images_target_test) and print accuracy.

#### YOUR CODE HERE ####


After fine tunning your model the accuracy should be higher now!

**This is the end of the exercise.**